[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MeteoSwiss/inca-examples/blob/main/notebooks/reprojecting_inca.ipynb)

# Reproject INCA data

In this example, we will show how to import and reproject a precipitation forecast from the INCA [nowcasting system](https://www.meteoswiss.admin.ch/home/services-and-publications/produkte.subpage.html/en/data/products/2021/nowcasting-inca-ch.html).

## Set-up Colab environment

**Important**: In colab, execute this section one cell at a time. Trying to excecute all the cells at once may results in cells being skipped and some dependencies not being installed.

First, let's set up our working environment. Note that these steps are only needed to work with google colab. 

#### Install dependencies

Now, let's install the  dependendies that will allow us to plot and read the example data.
- [xarray](https://github.com/pydata/xarray): work with labelled multi-dimensional arrays.
- [pyproj:](https://github.com/pyproj4/pyproj): cartographic projections and coordinate transformations.
- [rioxarray](https://github.com/corteva/rioxarray): rasterio xarray extension for geospatial datasets.

In [ ]:
!pip install xarray pyproj rioxarray

In [ ]:
!pip install xarray pyproj rioxarray

## Download the data

Now that we have the environment ready, let's download an example dataset from [Zenodo](https://zenodo.org/record/5575197) which includes the output of a single run of the [INCA nowcasting system](https://www.meteoswiss.admin.ch/home/services-and-publications/produkte.subpage.html/en/data/products/2021/nowcasting-inca-ch.html) for variable precipitation (mm/h).

In [ ]:
!wget https://zenodo.org/record/5575197/files/RR_INCA.nc

## Read the data

Now that we have downloaded the example data, let's import the example netCDF with xarray's `open_dataset` method.

We import a 6-hour precipitation nowcast, which corresponds to a sequence of 37 frames of 2-D precipitation composites (1 analysis and 36 forecast lead times).

In [ ]:
import xarray as xr

ds = xr.open_dataset("RR_INCA.nc")
ds

## Reproject with pyproj

The Swiss CH1903 / LV95 coordinates ([EPSG 21781]((https://epsg.io/21781)) can be reprojected to any other reference system using pyproj. 
In the example below, we'll use WGS84 as the target projection ([EPSG 4326](https://epsg.io/4326)).

Note that as a result, the dataset won't be anymore on a regular grid.

In [ ]:
import numpy as np
from pyproj import Transformer

chxv, chyv = np.meshgrid(ds.chx, ds.chy)
transformer = Transformer.from_crs(
    "epsg:21781", 
    "epsg:4326",
    always_xy=True,
) 
lon, lat = transformer.transform(chxv, chyv)
ds = ds.assign_coords({"lon": (("chy", "chx"), lon) , "lat": (("chy", "chx"), lat)})
ds.RR.isel(time=0).plot.pcolormesh("lon", "lat")  # not a regular grid anymore

## Reproject and resample with rioxarray

The rioxarray library extends xarray with methods based on rasterio to handle georefenced dataset. This makes it very easy to reproject and resample datasets between different CRS.

In [ ]:
import rioxarray # for the extension to load

ds = xr.open_dataset("RR_INCA.nc")
ds = ds.rio.write_crs("epsg:21781")
ds_latlon = ds.rio.reproject("epsg:4326", resolution=0.01)
ds_latlon.RR.isel(time=0).plot()  # still a regular grid
ds_latlon